## Which vectorization?

In [1]:
!pip install mlflow boto3 dagshub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 733.8/733.8 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
# !aws configure

In [2]:
import mlflow
import dagshub

dagshub.init(repo_owner='djnold', repo_name='mlops-project', mlflow=True)

# Step 2: Set up the MLflow tracking server
mlflow.set_tracking_uri("https://dagshub.com/djnold/mlops-project.mlflow")

# Step 2: Set up the MLflow tracking server
# mlflow.set_tracking_uri("http://ec2-54-175-41-29.compute-1.amazonaws.com:5000/")

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=700c6d4c-4834-4cda-aaf3-445f15d9308c&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=cc52eaa7e4ab555b17862ed471244ef7874e6d6e85cadf38750589dd376d7c5f




Output()

Accessing as djnold

Initialized MLflow to track repo "djnold/mlops-project"

Repository djnold/mlops-project initialized!

In [3]:
# Set or create an experiment
mlflow.set_experiment("Exp 3 - TfIdf Trigram max_features")

2025/07/08 13:57:23 INFO mlflow.tracking.fluent: Experiment with name 'Exp 3 - TfIdf Trigram max_features' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/15d0e6fb55ce40cda55489fab6b773d4', creation_time=1751983043287, experiment_id='3', last_update_time=1751983043287, lifecycle_stage='active', name='Exp 3 - TfIdf Trigram max_features', tags={}>

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
import joblib

In [5]:
df = pd.read_csv('/content/mental_preprocessing.csv').dropna(subset=['statement'])
df.shape

(42389, 2)

In [6]:
# Step 1: Function to run the experiment
def run_experiment_tfidf_max_features(max_features):
    ngram_range = (1, 3)  # Trigram setting

    # Step 2: Vectorization using TF-IDF with varying max_features
    vectorizer_name = "tfidf"
    vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)

    X_train, X_test, y_train, y_test = train_test_split(df['statement'], df['sentiment'], test_size=0.2, random_state=42, stratify=df['sentiment'])

    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)

    # Step 4: Define and train a Random Forest model
    with mlflow.start_run() as run:
        # Set tags for the experiment and run
        mlflow.set_tag("mlflow.runName", f"TFIDF_Trigrams_max_features_{max_features}")
        mlflow.set_tag("experiment_type", "feature_engineering")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # Add a description
        mlflow.set_tag("description", f"RandomForest with TF-IDF Trigrams, max_features={max_features}")

        # Log vectorizer parameters
        mlflow.log_param("vectorizer_type", "TF-IDF")
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", max_features)

        # Log Random Forest parameters
        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        # Initialize and train the model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        model.fit(X_train, y_train)

        # Step 5: Make predictions and log metrics
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: TF-IDF Trigrams, max_features={max_features}")
        plt.savefig("confusion_matrix.png")
        mlflow.log_artifact("confusion_matrix.png")
        plt.close()

        # save the model
        # mlflow.sklearn.log_model(model, f"random_forest_model_tfidf_trigrams_{max_features}")
        joblib.dump(model, f'/content/model.joblib.random_forest_model_{vectorizer_name}_{ngram_range}')

# Step 6: Test various max_features values
max_features_values = [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]

for max_features in max_features_values:
    run_experiment_tfidf_max_features(max_features)


🏃 View run TFIDF_Trigrams_max_features_1000 at: https://dagshub.com/djnold/mlops-project.mlflow/#/experiments/3/runs/aec46a21f9b7401fad970adcfe9c952b
🧪 View experiment at: https://dagshub.com/djnold/mlops-project.mlflow/#/experiments/3
🏃 View run TFIDF_Trigrams_max_features_2000 at: https://dagshub.com/djnold/mlops-project.mlflow/#/experiments/3/runs/b722d36aeb5449768288e635ca32f5aa
🧪 View experiment at: https://dagshub.com/djnold/mlops-project.mlflow/#/experiments/3
🏃 View run TFIDF_Trigrams_max_features_3000 at: https://dagshub.com/djnold/mlops-project.mlflow/#/experiments/3/runs/b655c336e254462fbb70697024f2eb0d
🧪 View experiment at: https://dagshub.com/djnold/mlops-project.mlflow/#/experiments/3
🏃 View run TFIDF_Trigrams_max_features_4000 at: https://dagshub.com/djnold/mlops-project.mlflow/#/experiments/3/runs/fe51e2757ecc4da786afd5902f389af9
🧪 View experiment at: https://dagshub.com/djnold/mlops-project.mlflow/#/experiments/3
🏃 View run TFIDF_Trigrams_max_features_5000 at: https://